Connected to Python 3.13.7

# MATTEO OLIVER BEKINK

In [44]:
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_digits
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF, WhiteKernel,ConstantKernel, ExpSineSquared, DotProduct, Matern, RationalQuadratic
from sklearn.ensemble import RandomForestClassifier

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

from data_viz import read_data

I will try to fit a variety of classification methods to find the best one:
* Random Forest
* Gaussian Classifier
* Gaussian Data Model

#### DATA PREP

In [69]:
train_data = read_data('train_set.csv')
test_data = read_data('test_set.csv')

train_data.head()

Data directory: C:\Users\matte\OneDrive\Documents\UC3M\UC3M_Notes\Machine_Learning II\Kaggle_Competition\data\train_set.csv
Data directory: C:\Users\matte\OneDrive\Documents\UC3M\UC3M_Notes\Machine_Learning II\Kaggle_Competition\data\test_set.csv


,target,feat_0,feat_1,feat_2,feat_3,feat_4,feat_5,feat_6,feat_7,feat_8,...,feat_17,feat_18,feat_19,feat_20,feat_21,feat_22,feat_23,feat_24,feat_25,feat_26
ID,,,,,,,,,,,,,,,,,,,,,
0,True,0.2616,2.5025,-11.1660,-0.3815,0.2319,-1.4798,0.0072,-4.1052,0.5083,...,-4.9826,-12.1896,1.3346,-0.1038,-0.1328,-0.2646,11.5052,0.1535,0.0963,0.3345
1,True,0.3874,1.0461,-9.4303,0.0100,-0.9858,-0.4959,0.0044,-0.3943,1.0242,...,-1.9471,-8.3548,6.0248,0.0607,-0.1689,-0.1911,9.4314,0.1386,3.7278,-0.5929
2,True,0.2276,5.5317,-10.4569,0.1915,-0.8280,-0.2836,0.0052,-3.8544,0.9625,...,1.2284,-0.8834,2.4716,0.2844,0.0231,-0.4979,10.3966,0.1455,-1.1622,-0.2638
3,False,0.3544,-0.2149,-3.9802,0.1635,0.5326,-0.4676,0.0268,3.0157,-1.4849,...,NaN,0.9018,0.2686,0.0460,0.2039,-2.2213,3.6265,-0.1575,1.5227,4.7911
4,False,0.3118,3.2238,-5.9532,-0.0805,1.1114,1.3232,0.0190,3.4671,-0.7132,...,-0.7276,-2.2058,-3.4741,0.2296,-0.1185,1.3222,6.1321,0.3408,1.8497,-0.9679


In [46]:
print(train_data.columns)

Index(['target', 'feat_0', 'feat_1', 'feat_2', 'feat_3', 'feat_4', 'feat_5',
       'feat_6', 'feat_7', 'feat_8', 'feat_9', 'feat_10', 'feat_11', 'feat_12',
       'feat_13', 'feat_14', 'feat_15', 'feat_16', 'feat_17', 'feat_18',
       'feat_19', 'feat_20', 'feat_21', 'feat_22', 'feat_23', 'feat_24',
       'feat_25', 'feat_26'],
      dtype='object')


In [70]:
print("Any missing values in train data:", train_data.isnull().any().any())
print("Any missing values in test data:", test_data.isnull().any().any())

clean_train_data = train_data.dropna()
clean_test_data = test_data.dropna()

Any missing values in train data: True
Any missing values in test data: False


In [58]:
def binary_classification_problem(train_data, test_data):
    X_train = train_data.drop(columns='target')
    y_train = train_data['target']

    # Test data doesn't have 'target' column (it's what we need to predict)
    X_test = test_data
    
    return X_train, y_train, X_test

X_train, y_train, X_test = binary_classification_problem(clean_train_data, clean_test_data)

### RANDOM FOREST

In [63]:
def random_forest_classification(X_train, y_train):
    mod = RandomForestClassifier(n_estimators=100, max_leaf_nodes=50, random_state=42)
    mod.fit(X_train, y_train)

    # Create relevance array
    feature_relevance_rf = np.zeros(X_train.shape[1])
    for tree in mod.estimators_:
        feature_relevance_rf += tree.feature_importances_
    feature_relevance_rf /= mod.n_estimators

    sorted_indices_rf_79 = np.argsort(feature_relevance_rf)[::-1]
    print("Feature ranking (Random Forest):")
    for rank, index in enumerate(sorted_indices_rf_79):
        print(f"{rank + 1}. Feature {index} (Relevance: {feature_relevance_rf[index]:.4f})")

    print(f"Sum of feature relevances: {np.sum(feature_relevance_rf):.4f}")
    return mod, feature_relevance_rf

Prediction

In [64]:
if not os.path.isdir('../outputs'):
    os.makedirs('../outputs')
mod, feature_relevance_rf = random_forest_classification(X_train, y_train)
predictions = mod.predict(X_test)
output_df = pd.DataFrame({'ID': test_data.index, 'target': predictions})
output_df.to_csv('../outputs/rf_predictions_with_na.csv', index=False)

Feature ranking (Random Forest):
1. Feature 9 (Relevance: 0.1551)
2. Feature 6 (Relevance: 0.1268)
3. Feature 11 (Relevance: 0.0874)
4. Feature 13 (Relevance: 0.0788)
5. Feature 5 (Relevance: 0.0742)
6. Feature 19 (Relevance: 0.0683)
7. Feature 16 (Relevance: 0.0536)
8. Feature 15 (Relevance: 0.0481)
9. Feature 8 (Relevance: 0.0268)
10. Feature 4 (Relevance: 0.0262)
11. Feature 10 (Relevance: 0.0257)
12. Feature 23 (Relevance: 0.0230)
13. Feature 18 (Relevance: 0.0226)
14. Feature 0 (Relevance: 0.0193)
15. Feature 14 (Relevance: 0.0170)
16. Feature 2 (Relevance: 0.0164)
17. Feature 24 (Relevance: 0.0145)
18. Feature 25 (Relevance: 0.0136)
19. Feature 3 (Relevance: 0.0132)
20. Feature 26 (Relevance: 0.0127)
21. Feature 22 (Relevance: 0.0120)
22. Feature 21 (Relevance: 0.0115)
23. Feature 7 (Relevance: 0.0115)
24. Feature 20 (Relevance: 0.0114)
25. Feature 1 (Relevance: 0.0104)
26. Feature 17 (Relevance: 0.0101)
27. Feature 12 (Relevance: 0.0097)
Sum of feature relevances: 1.0000


### GAUSSIAN CLASSIFIER - ARD - NOISE

In [ ]:
def gaussian_process_classification(X_train, y_train,length_scale_init=2.0):
    kernel = ConstantKernel(1.0) * RBF(length_scale=[length_scale_init] * 27, length_scale_bounds=[(1e-3, 1e10)] * 27) + WhiteKernel(noise_level=1, noise_level_bounds=(1e-5, 1e5))

    print('Preparing Gaussian Process Classifier...')
    mod = GaussianProcessClassifier(kernel=kernel, random_state=42, n_restarts_optimizer=2)
    print('Fitting Gaussian Process Classifier...')
    mod.fit(X_train, y_train)
    print('Gaussian Process Classifier fitted.')

    # Debug: Print all kernel parameters
    print("\nKernel parameters:")
    print(mod.kernel_.get_params())

    # Extract lengthscales - adjust key based on kernel structure
    kernel_params = mod.kernel_.get_params()
    # The structure is: k1__k2__length_scale (ConstantKernel * RBF) + WhiteKernel
    lengthscales_gp = kernel_params.get('k1__k2__length_scale', None)

    if lengthscales_gp is None:
        print("Trying alternative key...")
        lengthscales_gp = kernel_params.get('k2__length_scale', None)

    if lengthscales_gp is None:
        print("ERROR: Could not find lengthscales. Available keys:")
        for key in kernel_params.keys():
            print(f"  - {key}")
    else:
        # Sort in descending order of relevance (ascending order of lengthscales)
        sorted_indices_gp = np.argsort(lengthscales_gp)

        # Print the feature ranking
        print("Feature ranking (most to least relevant):")
        for rank, index in enumerate(sorted_indices_gp[:10]):  # Show top 10 most relevant features
            print(f"{rank + 1}. Feature {index} (Lengthscale: {lengthscales_gp[index]:.4f})")
    return mod, lengthscales_gp


Prediction

In [ ]:
if not os.path.isdir('../outputs'):
    os.makedirs('../outputs')
mod, lengthscales_gp = gaussian_process_classification(X_train, y_train)
predictions = mod.predict(X_test)
output_df = pd.DataFrame({'ID': test_data.index, 'target': predictions})
output_df.to_csv('../outputs/gp_predictions_with_na.csv', index=False)

## FULL IMPUTED DATA SET

In [86]:
train_data = read_data('full_train_set_imputed.csv')
test_data = read_data('test_set.csv')

train_data.head()

Data directory: C:\Users\matte\OneDrive\Documents\UC3M\UC3M_Notes\Machine_Learning II\Kaggle_Competition\data\full_train_set_imputed.csv
Data directory: C:\Users\matte\OneDrive\Documents\UC3M\UC3M_Notes\Machine_Learning II\Kaggle_Competition\data\test_set.csv


,target,feat_0,feat_1,feat_2,feat_3,feat_4,feat_5,feat_6,feat_7,feat_8,...,feat_17,feat_18,feat_19,feat_20,feat_21,feat_22,feat_23,feat_24,feat_25,feat_26
ID,,,,,,,,,,,,,,,,,,,,,
0,True,0.2616,2.5025,-11.1660,-0.3815,0.2319,-1.4798,0.0072,-4.1052,0.5083,...,-4.982600,-12.1896,1.3346,-0.1038,-0.1328,-0.2646,11.5052,0.1535,0.0963,0.3345
1,True,0.3874,1.0461,-9.4303,0.0100,-0.9858,-0.4959,0.0044,-0.3943,1.0242,...,-1.947100,-8.3548,6.0248,0.0607,-0.1689,-0.1911,9.4314,0.1386,3.7278,-0.5929
2,True,0.2276,5.5317,-10.4569,0.1915,-0.8280,-0.2836,0.0052,-3.8544,0.9625,...,1.228400,-0.8834,2.4716,0.2844,0.0231,-0.4979,10.3966,0.1455,-1.1622,-0.2638
3,False,0.3544,-0.2149,-3.9802,0.1635,0.5326,-0.4676,0.0268,3.0157,-1.4849,...,-0.652831,0.9018,0.2686,0.0460,0.2039,-2.2213,3.6265,-0.1575,1.5227,4.7911
4,False,0.3118,3.2238,-5.9532,-0.0805,1.1114,1.3232,0.0190,3.4671,-0.7132,...,-0.727600,-2.2058,-3.4741,0.2296,-0.1185,1.3222,6.1321,0.3408,1.8497,-0.9679


In [87]:
print("Any missing values in train data:", train_data.isnull().any().any())
print("Any missing values in test data:", test_data.isnull().any().any())

Any missing values in train data: False
Any missing values in test data: False


In [88]:
X_train, y_train, X_test = binary_classification_problem(train_data, test_data)

## RANDOM FOREST 

In [85]:
if not os.path.isdir('../outputs'):
    os.makedirs('../outputs')
mod, feature_relevance_rf = random_forest_classification(X_train, y_train)
predictions = mod.predict(X_test)
output_df = pd.DataFrame({'ID': test_data.index, 'target': predictions})
output_df.to_csv('../outputs/rf_predictions_without_na.csv', index=False)

Feature ranking (Random Forest):
1. Feature 9 (Relevance: 0.1635)
2. Feature 6 (Relevance: 0.1286)
3. Feature 11 (Relevance: 0.1033)
4. Feature 5 (Relevance: 0.0795)
5. Feature 13 (Relevance: 0.0784)
6. Feature 19 (Relevance: 0.0692)
7. Feature 16 (Relevance: 0.0562)
8. Feature 15 (Relevance: 0.0387)
9. Feature 4 (Relevance: 0.0321)
10. Feature 18 (Relevance: 0.0262)
11. Feature 8 (Relevance: 0.0257)
12. Feature 10 (Relevance: 0.0250)
13. Feature 23 (Relevance: 0.0206)
14. Feature 2 (Relevance: 0.0181)
15. Feature 0 (Relevance: 0.0141)
16. Feature 14 (Relevance: 0.0130)
17. Feature 26 (Relevance: 0.0125)
18. Feature 24 (Relevance: 0.0110)
19. Feature 25 (Relevance: 0.0104)
20. Feature 3 (Relevance: 0.0102)
21. Feature 22 (Relevance: 0.0101)
22. Feature 7 (Relevance: 0.0101)
23. Feature 21 (Relevance: 0.0097)
24. Feature 12 (Relevance: 0.0091)
25. Feature 20 (Relevance: 0.0090)
26. Feature 17 (Relevance: 0.0081)
27. Feature 1 (Relevance: 0.0076)
Sum of feature relevances: 1.0000


## GAUSSIAN CLASSIFIER

In [81]:
mod, lengthscales_gp = gaussian_process_classification(X_train, y_train)
if not os.path.isdir('../outputs'):
    os.makedirs('../outputs')
predictions = mod.predict(X_test)
output_df = pd.DataFrame({'ID': test_data.index, 'target': predictions})
output_df.to_csv('../outputs/gp_predictions_without_na_v2.csv', index=False)


Preparing Gaussian Process Classifier...
Fitting Gaussian Process Classifier...


c:\Users\matte\.conda\envs\Optimization_Analytics\Lib\site-packages\sklearn\gaussian_process\kernels.py:440: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(


Gaussian Process Classifier fitted.

Kernel parameters:
{'k1': 3.91**2 * RBF(length_scale=[2.11, 2.85e+04, 1.44e+07, 3.72, 6.89, 6.09, 0.0244, 9.37e+04, 7.68, 1.17, 4.43, 0.0388, 1.07e+04, 0.912, 217, 6.91, 2.04, 1.08e+04, 32.6, 10.9, 614, 864, 5.38e+04, 18.6, 4.54, 4.85e+04, 6.14e+04]), 'k2': WhiteKernel(noise_level=1e-05), 'k1__k1': 3.91**2, 'k1__k2': RBF(length_scale=[2.11, 2.85e+04, 1.44e+07, 3.72, 6.89, 6.09, 0.0244, 9.37e+04, 7.68, 1.17, 4.43, 0.0388, 1.07e+04, 0.912, 217, 6.91, 2.04, 1.08e+04, 32.6, 10.9, 614, 864, 5.38e+04, 18.6, 4.54, 4.85e+04, 6.14e+04]), 'k1__k1__constant_value': np.float64(15.263042146961178), 'k1__k1__constant_value_bounds': (1e-05, 100000.0), 'k1__k2__length_scale': array([2.11208124e+00, 2.84544260e+04, 1.44312322e+07, 3.72174727e+00,
       6.88548058e+00, 6.08598276e+00, 2.43841489e-02, 9.36954364e+04,
       7.67670793e+00, 1.16591643e+00, 4.43189034e+00, 3.88156692e-02,
       1.06922517e+04, 9.12130746e-01, 2.16770599e+02, 6.91247675e+00,
       2.0

## GAUSSIAN CLASSFIER WITH SCALING

In [89]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

mod, lengthscales_gp = gaussian_process_classification(X_train_scaled, y_train)
if not os.path.isdir('../outputs'):
    os.makedirs('../outputs')
predictions = mod.predict(X_test_scaled)
output_df = pd.DataFrame({'ID': test_data.index, 'target': predictions})
output_df.to_csv('../outputs/gp_predictions_without_na_scaled.csv', index=False)

Preparing Gaussian Process Classifier...
Fitting Gaussian Process Classifier...


c:\Users\matte\.conda\envs\Optimization_Analytics\Lib\site-packages\sklearn\gaussian_process\kernels.py:450: ConvergenceWarning: The optimal value found for dimension 2 of parameter k1__k2__length_scale is close to the specified upper bound 10000000000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
c:\Users\matte\.conda\envs\Optimization_Analytics\Lib\site-packages\sklearn\gaussian_process\kernels.py:440: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(


Gaussian Process Classifier fitted.

Kernel parameters:
{'k1': 3.91**2 * RBF(length_scale=[15.1, 4.55e+03, 1e+10, 14.5, 7.84, 6.05, 2.94, 5.13e+03, 9.76, 3.97, 3.01, 3.87, 4.41e+03, 4.82, 3.17e+03, 5.01, 3.14, 5.07e+03, 3.17, 2.68, 1.48e+03, 5.32e+03, 5.01e+03, 5.17, 25.2, 897, 4.3e+03]), 'k2': WhiteKernel(noise_level=1e-05), 'k1__k1': 3.91**2, 'k1__k2': RBF(length_scale=[15.1, 4.55e+03, 1e+10, 14.5, 7.84, 6.05, 2.94, 5.13e+03, 9.76, 3.97, 3.01, 3.87, 4.41e+03, 4.82, 3.17e+03, 5.01, 3.14, 5.07e+03, 3.17, 2.68, 1.48e+03, 5.32e+03, 5.01e+03, 5.17, 25.2, 897, 4.3e+03]), 'k1__k1__constant_value': np.float64(15.260027129157654), 'k1__k1__constant_value_bounds': (1e-05, 100000.0), 'k1__k2__length_scale': array([1.51070230e+01, 4.54682656e+03, 1.00000000e+10, 1.45234488e+01,
       7.83945116e+00, 6.04906342e+00, 2.94287331e+00, 5.12779041e+03,
       9.76413621e+00, 3.96803052e+00, 3.00748248e+00, 3.86897500e+00,
       4.40996260e+03, 4.81962795e+00, 3.16876623e+03, 5.01127248e+00,
       3